In [1]:
from __future__ import print_function, division
import scipy

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys
import os

import numpy as np

class COGAN():
    """Reference: https://wiseodd.github.io/techblog/2017/02/18/coupled_gan/"""
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.d1, self.d2 = self.build_discriminators()
        self.d1.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])
        self.d2.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.g1, self.g2 = self.build_generators()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img1 = self.g1(z)
        img2 = self.g2(z)

        # For the combined model we will only train the generators
        self.d1.trainable = False
        self.d2.trainable = False

        # The valid takes generated images as input and determines validity
        valid1 = self.d1(img1)
        valid2 = self.d2(img2)

        # The combined model  (stacked generators and discriminators)
        # Trains generators to fool discriminators
        self.combined = Model(z, [valid1, valid2])
        self.combined.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
                                    optimizer=optimizer)

    def build_generators(self):

        # Shared weights between generators
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        noise = Input(shape=(self.latent_dim,))
        feature_repr = model(noise)

        # Generator 1
        g1 = Dense(1024)(feature_repr)
        g1 = LeakyReLU(alpha=0.2)(g1)
        g1 = BatchNormalization(momentum=0.8)(g1)
        g1 = Dense(np.prod(self.img_shape), activation='tanh')(g1)
        img1 = Reshape(self.img_shape)(g1)

        # Generator 2
        g2 = Dense(1024)(feature_repr)
        g2 = LeakyReLU(alpha=0.2)(g2)
        g2 = BatchNormalization(momentum=0.8)(g2)
        g2 = Dense(np.prod(self.img_shape), activation='tanh')(g2)
        img2 = Reshape(self.img_shape)(g2)

        model.summary()

        return Model(noise, img1), Model(noise, img2)

    def build_discriminators(self):

        img1 = Input(shape=self.img_shape)
        img2 = Input(shape=self.img_shape)

        # Shared discriminator layers
        model = Sequential()
        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))

        img1_embedding = model(img1)
        img2_embedding = model(img2)

        # Discriminator 1
        validity1 = Dense(1, activation='sigmoid')(img1_embedding)
        # Discriminator 2
        validity2 = Dense(1, activation='sigmoid')(img2_embedding)

        return Model(img1, validity1), Model(img2, validity2)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Images in domain A and B (rotated)
        X1 = X_train[:int(X_train.shape[0]/2)]
        X2 = X_train[int(X_train.shape[0]/2):]
        X2 = scipy.ndimage.interpolation.rotate(X2, 90, axes=(1, 2))

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                # imgs = X_train[index * batch_size:(index + 1) * batch_size]

                imgs1 = X1[index * batch_size:(index + 1) * batch_size]
                imgs2 = X2[index * batch_size:(index + 1) * batch_size]

                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, 100))

                # Generate a batch of new images
                gen_imgs1 = self.g1.predict(noise)
                gen_imgs2 = self.g2.predict(noise)

                # Train the discriminators
                d1_loss_real = self.d1.train_on_batch(imgs1, valid)
                d2_loss_real = self.d2.train_on_batch(imgs2, valid)
                d1_loss_fake = self.d1.train_on_batch(gen_imgs1, fake)
                d2_loss_fake = self.d2.train_on_batch(gen_imgs2, fake)
                d1_loss = 0.5 * np.add(d1_loss_real, d1_loss_fake)
                d2_loss = 0.5 * np.add(d2_loss_real, d2_loss_fake)

                # ------------------
                #  Train Generators
                # ------------------

                g_loss = self.combined.train_on_batch(noise, [valid, valid])

                # Plot the progress
                print("epoch:%d step:%d [D1 loss: %f, acc.: %.2f%%] [D2 loss: %f, acc.: %.2f%%] [G loss: %f]" \
                      % (epoch,global_step, d1_loss[0], 100 * d1_loss[1], d2_loss[0], 100 * d2_loss[1], g_loss[0]))

                # If at save interval => save generated image samples
                if  global_step % sample_interval == 0:
                    self.sample_images(epoch, global_step)

    def sample_images(self, epoch,global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * int(c/2), 100))
        gen_imgs1 = self.g1.predict(noise)
        gen_imgs2 = self.g2.predict(noise)

        gen_imgs = np.concatenate([gen_imgs1, gen_imgs2])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        if not os.path.isdir('images_cogan'):
            os.mkdir('images_cogan')
        fig.savefig("images_cogan/epoch_%d_step_%d.png" % (epoch,global_step))
        plt.close()


if __name__ == '__main__':
    gan = COGAN()
    gan.train(epochs=30, batch_size=64, sample_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_6 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
Total params: 160,512
Trainable params: 158,976
Non-trainable params: 1,536
_________________________________________________________________


/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D1 loss: 0.569182, acc.: 51.56%] [D2 loss: 0.589382, acc.: 77.34%] [G loss: 1.364645]
epoch:0 step:2 [D1 loss: 0.478858, acc.: 61.72%] [D2 loss: 0.381388, acc.: 75.00%] [G loss: 1.643843]
epoch:0 step:3 [D1 loss: 0.332318, acc.: 78.91%] [D2 loss: 0.282622, acc.: 88.28%] [G loss: 2.116379]
epoch:0 step:4 [D1 loss: 0.264059, acc.: 92.97%] [D2 loss: 0.200256, acc.: 98.44%] [G loss: 2.728363]
epoch:0 step:5 [D1 loss: 0.169261, acc.: 98.44%] [D2 loss: 0.161291, acc.: 100.00%] [G loss: 3.325578]
epoch:0 step:6 [D1 loss: 0.139687, acc.: 100.00%] [D2 loss: 0.137714, acc.: 100.00%] [G loss: 3.655188]
epoch:0 step:7 [D1 loss: 0.123926, acc.: 100.00%] [D2 loss: 0.107207, acc.: 100.00%] [G loss: 4.060299]
epoch:0 step:8 [D1 loss: 0.097487, acc.: 100.00%] [D2 loss: 0.081975, acc.: 100.00%] [G loss: 4.439066]
epoch:0 step:9 [D1 loss: 0.078853, acc.: 100.00%] [D2 loss: 0.066776, acc.: 100.00%] [G loss: 4.690831]
epoch:0 step:10 [D1 loss: 0.070649, acc.: 100.00%] [D2 loss: 0.071406, ac

epoch:0 step:81 [D1 loss: 0.042361, acc.: 98.44%] [D2 loss: 0.031608, acc.: 100.00%] [G loss: 10.590966]
epoch:0 step:82 [D1 loss: 0.059716, acc.: 97.66%] [D2 loss: 0.042152, acc.: 100.00%] [G loss: 10.774583]
epoch:0 step:83 [D1 loss: 0.144527, acc.: 95.31%] [D2 loss: 0.129568, acc.: 93.75%] [G loss: 10.120764]
epoch:0 step:84 [D1 loss: 0.023194, acc.: 100.00%] [D2 loss: 0.035554, acc.: 98.44%] [G loss: 10.665756]
epoch:0 step:85 [D1 loss: 0.186609, acc.: 92.97%] [D2 loss: 0.081559, acc.: 96.88%] [G loss: 10.070204]
epoch:0 step:86 [D1 loss: 0.032954, acc.: 99.22%] [D2 loss: 0.067124, acc.: 98.44%] [G loss: 10.712704]
epoch:0 step:87 [D1 loss: 0.081205, acc.: 96.88%] [D2 loss: 0.043208, acc.: 100.00%] [G loss: 10.925547]
epoch:0 step:88 [D1 loss: 0.062684, acc.: 97.66%] [D2 loss: 0.058795, acc.: 99.22%] [G loss: 10.400395]
epoch:0 step:89 [D1 loss: 0.057572, acc.: 100.00%] [D2 loss: 0.041386, acc.: 99.22%] [G loss: 10.475477]
epoch:0 step:90 [D1 loss: 0.070739, acc.: 98.44%] [D2 loss:

epoch:0 step:161 [D1 loss: 0.669311, acc.: 60.16%] [D2 loss: 0.587113, acc.: 65.62%] [G loss: 5.665946]
epoch:0 step:162 [D1 loss: 0.467688, acc.: 75.00%] [D2 loss: 0.341282, acc.: 87.50%] [G loss: 7.388217]
epoch:0 step:163 [D1 loss: 0.623842, acc.: 62.50%] [D2 loss: 0.432358, acc.: 74.22%] [G loss: 6.129644]
epoch:0 step:164 [D1 loss: 0.664074, acc.: 57.81%] [D2 loss: 0.603922, acc.: 64.84%] [G loss: 5.716417]
epoch:0 step:165 [D1 loss: 0.557528, acc.: 64.06%] [D2 loss: 0.499284, acc.: 71.09%] [G loss: 4.852267]
epoch:0 step:166 [D1 loss: 0.518499, acc.: 69.53%] [D2 loss: 0.446361, acc.: 71.09%] [G loss: 7.125200]
epoch:0 step:167 [D1 loss: 0.826796, acc.: 43.75%] [D2 loss: 0.539961, acc.: 60.16%] [G loss: 7.455234]
epoch:0 step:168 [D1 loss: 0.730589, acc.: 53.12%] [D2 loss: 0.422031, acc.: 75.78%] [G loss: 6.404889]
epoch:0 step:169 [D1 loss: 0.803150, acc.: 50.78%] [D2 loss: 0.908095, acc.: 46.88%] [G loss: 2.372348]
epoch:0 step:170 [D1 loss: 0.416418, acc.: 71.88%] [D2 loss: 0.3

epoch:0 step:241 [D1 loss: 0.663986, acc.: 51.56%] [D2 loss: 0.677443, acc.: 43.75%] [G loss: 1.544929]
epoch:0 step:242 [D1 loss: 0.675900, acc.: 50.78%] [D2 loss: 0.652991, acc.: 46.88%] [G loss: 1.570542]
epoch:0 step:243 [D1 loss: 0.686960, acc.: 45.31%] [D2 loss: 0.643290, acc.: 51.56%] [G loss: 1.594382]
epoch:0 step:244 [D1 loss: 0.670233, acc.: 52.34%] [D2 loss: 0.616593, acc.: 59.38%] [G loss: 1.708507]
epoch:0 step:245 [D1 loss: 0.700490, acc.: 46.88%] [D2 loss: 0.666527, acc.: 47.66%] [G loss: 1.578455]
epoch:0 step:246 [D1 loss: 0.709802, acc.: 43.75%] [D2 loss: 0.640521, acc.: 53.12%] [G loss: 1.524535]
epoch:0 step:247 [D1 loss: 0.674850, acc.: 50.78%] [D2 loss: 0.675551, acc.: 42.97%] [G loss: 1.558774]
epoch:0 step:248 [D1 loss: 0.686261, acc.: 46.09%] [D2 loss: 0.649815, acc.: 41.41%] [G loss: 1.608441]
epoch:0 step:249 [D1 loss: 0.661369, acc.: 56.25%] [D2 loss: 0.644855, acc.: 53.91%] [G loss: 1.659880]
epoch:0 step:250 [D1 loss: 0.662039, acc.: 56.25%] [D2 loss: 0.6

epoch:0 step:321 [D1 loss: 0.682210, acc.: 51.56%] [D2 loss: 0.652366, acc.: 50.78%] [G loss: 1.508982]
epoch:0 step:322 [D1 loss: 0.652270, acc.: 56.25%] [D2 loss: 0.629306, acc.: 55.47%] [G loss: 1.603310]
epoch:0 step:323 [D1 loss: 0.661815, acc.: 50.78%] [D2 loss: 0.660115, acc.: 50.00%] [G loss: 1.561544]
epoch:0 step:324 [D1 loss: 0.645338, acc.: 49.22%] [D2 loss: 0.637118, acc.: 50.00%] [G loss: 1.521414]
epoch:0 step:325 [D1 loss: 0.629386, acc.: 60.94%] [D2 loss: 0.634532, acc.: 64.06%] [G loss: 1.618562]
epoch:0 step:326 [D1 loss: 0.671205, acc.: 57.03%] [D2 loss: 0.649983, acc.: 56.25%] [G loss: 1.630757]
epoch:0 step:327 [D1 loss: 0.649121, acc.: 59.38%] [D2 loss: 0.593512, acc.: 66.41%] [G loss: 1.771948]
epoch:0 step:328 [D1 loss: 0.666776, acc.: 49.22%] [D2 loss: 0.661893, acc.: 47.66%] [G loss: 1.649515]
epoch:0 step:329 [D1 loss: 0.665746, acc.: 54.69%] [D2 loss: 0.641587, acc.: 52.34%] [G loss: 1.522134]
epoch:0 step:330 [D1 loss: 0.640105, acc.: 60.94%] [D2 loss: 0.6

epoch:0 step:401 [D1 loss: 0.679543, acc.: 50.00%] [D2 loss: 0.636688, acc.: 64.84%] [G loss: 1.644575]
epoch:0 step:402 [D1 loss: 0.652232, acc.: 59.38%] [D2 loss: 0.579638, acc.: 72.66%] [G loss: 1.681696]
epoch:0 step:403 [D1 loss: 0.677822, acc.: 48.44%] [D2 loss: 0.602913, acc.: 64.84%] [G loss: 1.742830]
epoch:0 step:404 [D1 loss: 0.686832, acc.: 47.66%] [D2 loss: 0.631338, acc.: 64.06%] [G loss: 1.835646]
epoch:0 step:405 [D1 loss: 0.675251, acc.: 53.12%] [D2 loss: 0.648254, acc.: 59.38%] [G loss: 1.639475]
epoch:0 step:406 [D1 loss: 0.660779, acc.: 57.81%] [D2 loss: 0.636940, acc.: 63.28%] [G loss: 1.597425]
epoch:0 step:407 [D1 loss: 0.680084, acc.: 51.56%] [D2 loss: 0.655920, acc.: 59.38%] [G loss: 1.586995]
epoch:0 step:408 [D1 loss: 0.681361, acc.: 50.00%] [D2 loss: 0.653595, acc.: 55.47%] [G loss: 1.636846]
epoch:0 step:409 [D1 loss: 0.665240, acc.: 58.59%] [D2 loss: 0.629121, acc.: 67.97%] [G loss: 1.676657]
epoch:0 step:410 [D1 loss: 0.662249, acc.: 52.34%] [D2 loss: 0.6

ValueError: Input arrays should have the same number of samples as target arrays. Found 48 input samples and 64 target samples.